In [1]:
import pl.mareklangiewicz.kommand.*
import pl.mareklangiewicz.kommand.CliPlatform.Companion.SYS
import pl.mareklangiewicz.kommand.core.*
import pl.mareklangiewicz.kommand.core.LsOpt.*
import pl.mareklangiewicz.kommand.gnome.*
import pl.mareklangiewicz.kommand.gnome.GnomeExt.Cmd.list

// TODO NOW: go through this whole notebook again (commented versions too) after refactor; also test input/output sequences (like slow find output)
// TODO NOW: analyze ProcessBuilder.startPipeline and understand assumptions/invariants and make sure my "pipelines" with line sequences are correct.
//    also reproduce example from ProcessBuilder.startPipeline javadoc using my stuff

SYS.run {
    start(gnometerm(bash(gnomeext(list), pause = true)))
    start(gnometerm(bash(ls { -LongFormat; -All }, pause = true)))
}




pl.mareklangiewicz.kommand.JvmExecProcess@599607f2

In [2]:
SYS.run {
    LsSamples.lsParentWithSlashes.exec().printlns()
}

build/
notebooks/
src/
build.gradle.kts


In [3]:
import pl.mareklangiewicz.kommand.konfig.*

SYS.konfigInUserHomeConfigDir().printAll()

    konfig["code.interactive.enabled"] == "true"
    konfig["README"] == "This is my default konfig directory with all kinds of settings. See KommandLine repo:Konfig.kt
TODO: migrate almost all my .bashrc / .profile settings to this directory.
This README file itself is also a value in the konfig (under "README" key),
but it will not be used as any setting obviously."
    konfig["tests.Langara.enabled"] == "true"
    konfig["tests.README"] == "Boolean flags under keys: tests.SimpleClassName.enabled allow to enable JUnit5 jvm tests like:
/home/marek/code/kotlin/KommandLine/kommanddemos/src/test/kotlin/demos/Langara.kt


In [5]:
import pl.mareklangiewicz.kommand.admin.*
import pl.mareklangiewicz.kommand.debian.*
import pl.mareklangiewicz.kommand.debian.DpkgAct.*
import pl.mareklangiewicz.kommand.samples.Samples

SYS.run {
    val kmd =
//        Samples.GitHub.help
//        Samples.GitHub.secretSetFakeSecretInAbcdK
//        Samples.GitHub.secretListForAbcdK
//        dpkg { -DpkgAct.PrintArch }
//        dpkg(Status("apt"))
//        dpkg(Details("findutils"))
//        sudo(bash("whoami; pwd; ls"))
//        sudo(kommand("aptitude"))
//        sudoEdit("/etc/hosts")
        sudoEdit("/etc/fstab")
//        kommand("whoami").withSudo(SudoOpt.User(zenityAskForEntryExec("Enter some user name")))
    startInGnomeTermIfUserConfirms(kmd)
}


In [6]:
import pl.mareklangiewicz.kommand.debian.*

SYS.dpkgSearchOneCommandExec("find")

[findutils: /usr/bin/find]

In [7]:
import pl.mareklangiewicz.kommand.admin.*
import pl.mareklangiewicz.kommand.find.*

SYS.run {
//    sudoExec(ls("/root/", withHidden = true), inPass = zenityAskForPasswordExec())
//    FindSamples.findAbcIgnoreCase.exec()
//    FindSamples.findSomeSamples.exec()
//    FindSamples.findNodeModulesDirs.exec().printlns()
//    FindSamples.findBuildDirs.exec().printlns()
//    FindSamples.findSymLinksToKtsFilesInDepsKt.exec().printlns()
//    FindSamples.findDepthMax2FilesInDepsKtAndRunFileOnEach.exec().printlns()
//    FindSamples.findInKotlinKtFilesModifiedIn24h.exec().printlns()
    FindSamples.findMyLastWeekKotlinCode.exec().printlns()
}

/home/marek/code/kotlin/KommandLine/kommandline/src/commonTest/kotlin/gnupg/GpgTest.kt
/home/marek/code/kotlin/KommandLine/kommandline/src/jvmMain/kotlin/JvmPlatform.kt
/home/marek/code/kotlin/KommandLine/kommandline/src/commonMain/kotlin/Zenity.kt
/home/marek/code/kotlin/KommandLine/kommandline/src/commonMain/kotlin/CliPlatform.kt
/home/marek/code/kotlin/KommandLine/kommandline/src/commonMain/kotlin/find/Find.kt
/home/marek/code/kotlin/KommandLine/kommandline/src/commonMain/kotlin/admin/Sudo.kt
/home/marek/code/kotlin/KommandLine/kommandline/src/commonMain/kotlin/Annotations.kt
/home/marek/code/kotlin/KommandLine/kommandline/src/commonMain/kotlin/Bash.kt
/home/marek/code/kotlin/KommandLine/kommandline/src/commonMain/kotlin/Kommand.kt
/home/marek/code/kotlin/KommandLine/kommandline/src/commonMain/kotlin/github/Gh.kt
/home/marek/code/kotlin/KommandLine/kommandline/src/commonMain/kotlin/debian/Which.kt
/home/marek/code/kotlin/KommandLine/kommandline/src/commonMain/kotlin/Utils.kt
/home/m

In [8]:
import pl.mareklangiewicz.kommand.find.*

SYS.findTypicalDetailsTableExec("..")


[[access time, status change time, last modification time, birth time, depth, size, dir name, base name, full name, group name, user name, octal permissions, symbolic permissions], [2023-07-11+13:23:35.5092699680, 2023-06-25+13:09:53.6910029450, 2023-06-25+13:09:53.6910029450, , 1, 12157, .., build.gradle.kts, ../build.gradle.kts, marek, marek, 664, -rw-rw-r--], [2023-07-11+16:03:37.4323810920, 2023-07-11+16:03:37.3243796230, 2023-07-11+16:03:37.3243796230, , 7, 140, ../build/classes/kotlin/jvm/test/META-INF, kommandsamples_test.kotlin_module, ../build/classes/kotlin/jvm/test/META-INF/kommandsamples_test.kotlin_module, marek, marek, 664, -rw-rw-r--], [2023-07-11+13:37:56.8901358030, 2023-07-11+13:37:56.8141347080, 2023-07-11+13:37:56.8141347080, , 10, 1314, ../build/classes/kotlin/jvm/test/pl/mareklangiewicz/kommand/samples, SamplesTestsKt$testSamplesObject$1.class, ../build/classes/kotlin/jvm/test/pl/mareklangiewicz/kommand/samples/SamplesTestsKt$testSamplesObject$1.class, marek, mare

In [9]:
import pl.mareklangiewicz.kommand.CliPlatform.Companion.SYS

SYS.run {
    findMyKotlinCode(
        withGrepRE = "@Test",
        withModifTime24h = FindExpr.NumArg.LessThan(8)
    ).checkInIdeap()
}

find /home/marek/code/kotlin \( \( -name build -type d -prune -false \) -o \( -path */src/*/kotlin/* -name *.kt -type f -mtime -8 \( -execdir grep -q @Test {} \; -print \) \) \)
